# High Price Predict Model

In [1]:
import pandas as pd
kospi = pd.read_csv('./KOSPI.csv')
kospi['종목코드'] = kospi['종목코드'].apply(lambda x : str(x).rjust(6, '0'))

code_list = kospi['종목코드'].unique()

In [2]:
submission = []

import numpy as np

for code in code_list : 

    tr_name = 'TR_SCHART'
    term = 'D'
    start_date = '20180510'   
    end_date = '20200229'
    Lookup = '9999' 

    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup

    from urllib.request import urlopen
    url_page = urlopen(url)

    import json
    url_data = json.loads(url_page.read())

    import pandas as pd
    data = pd.DataFrame(url_data)

    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr','rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]

    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
        
        
    
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
            
    data['y'] = data['high'].shift(-1)
    
    data = data.dropna()
    
    X = data[data.columns[:-1]]
    y = data['y']
    
    from sklearn.linear_model import LinearRegression
    reg = LinearRegression().fit(X, y)
    
    
    start_date = '20200301'   
    end_date = '20200316'
    url = 'http://ssecd.roboadvisor.co.kr:9999/' + tr_name + '?0=' + code + '&1=' + term + '&2=1' +\
          '&3=' + start_date + '&4=' + end_date + '&5=' + Lookup
    from urllib.request import urlopen
    url_page = urlopen(url)
    import json
    url_data = json.loads(url_page.read())
    import pandas as pd
    data = pd.DataFrame(url_data)
    data.columns = ['date','time','open','high','low','close','price_ccr','volume_ccr','rock','volume','volume_price']
    data = data[['date', 'time', 'open', 'high', 'low', 'close', 'volume', 'volume_price']]
    data = data[::-1]
    data = data.set_index('date')
    data = data[['open', 'high', 'low', 'close', 'volume', 'volume_price']]
    for column in data.columns : 
        data[column] = data[column].apply(lambda x : int(x))
    data['volume'] = data['volume'].apply(lambda x : np.log(x+1))
    data['volume_price'] = data['volume_price'].apply(lambda x : np.log(x+1))
    for column in ['open', 'high', 'low', 'close', 'volume', 'volume_price'] : 
        for i in range(1,5,1) : 
            data['{}_shift_{}'.format(column,i)] = data[column].shift(i)
    data['y'] = data['high'].shift(-1)
    data = data.dropna()
    data = data[-1:]
    data = data[data.columns[:-1]]

    print('{} 종목의 내일 고가 예측 가격 = {:,}원' .format(code, int(round(reg.predict(data)[0]))))
    
    pred = dict({"model" : "psh_lr_high",
              "seed" : 1234,
              "type" : "high",
              "code" : code,
              "date" : "20-03-16 00:00:00",
            "prdate" : 0,
             "value" : int(round(reg.predict(data)[0]))})
    
    submission.append(pred)

060310 종목의 내일 고가 예측 가격 = 2,086원
095570 종목의 내일 고가 예측 가격 = 3,560원
068400 종목의 내일 고가 예측 가격 = 7,898원
006840 종목의 내일 고가 예측 가격 = 19,389원
054620 종목의 내일 고가 예측 가격 = 6,965원
265520 종목의 내일 고가 예측 가격 = 25,556원
211270 종목의 내일 고가 예측 가격 = 6,066원
027410 종목의 내일 고가 예측 가격 = 3,846원
282330 종목의 내일 고가 예측 가격 = 136,644원
138930 종목의 내일 고가 예측 가격 = 4,879원
001460 종목의 내일 고가 예측 가격 = 178,508원
001040 종목의 내일 고가 예측 가격 = 62,006원
079160 종목의 내일 고가 예측 가격 = 21,881원
035760 종목의 내일 고가 예측 가격 = 105,040원
000120 종목의 내일 고가 예측 가격 = 123,188원
011150 종목의 내일 고가 예측 가격 = 3,061원
097950 종목의 내일 고가 예측 가격 = 223,322원
051500 종목의 내일 고가 예측 가격 = 16,498원
037560 종목의 내일 고가 예측 가격 = 3,259원
058820 종목의 내일 고가 예측 가격 = 3,269원
023460 종목의 내일 고가 예측 가격 = 1,163원
065770 종목의 내일 고가 예측 가격 = 1,922원
083660 종목의 내일 고가 예측 가격 = 4,170원
000590 종목의 내일 고가 예측 가격 = 49,065원
012030 종목의 내일 고가 예측 가격 = 625원
306620 종목의 내일 고가 예측 가격 = 1,674원
322780 종목의 내일 고가 예측 가격 = 2,046원
016610 종목의 내일 고가 예측 가격 = 3,033원
005830 종목의 내일 고가 예측 가격 = 30,389원
000990 종목의 내일 고가 예측 가격 = 21,241원
139130 종목의 내일 고가 예측 가격 =

ValueError: invalid literal for int() with base 10: ''

In [3]:
submission

[{'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '060310',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 2086},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '095570',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 3560},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '068400',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 7898},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '006840',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 19389},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '054620',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 6965},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '265520',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value': 25556},
 {'model': 'psh_lr_high',
  'seed': 1234,
  'type': 'high',
  'code': '211270',
  'date': '20-03-16 00:00:00',
  'prdate': 0,
  'value

In [ ]:
# import requests
# import json

# def send_predicts(predict):
#     payload = json.dumps(predict)
#     print(payload)
#     url = "https://lab.donutz.co/tensors"
#     headers = {'Content-Type': 'application/json; charset=utf-8'}
#     #response = requests.get(url, params=paramDict)
#     response = requests.post(url, data=payload, headers=headers, verify=False)
#     code = response.status_code
#     res = response.text
#     return (code, res)

# predict = pred
# print(send_predicts(predict))